In [20]:
from pathlib import Path
import s3fs
import xarray as xr

In [21]:
s3cfg = dict([l.split(' = ') for l in Path('/home/users/mmuetz/.s3cfg').read_text().split('\n') if l])

In [22]:
# Internal access allowed in notebooks - no https and s3-ext.
jasmin_s3 = s3fs.S3FileSystem(
    anon=False, secret=s3cfg['secret_key'],
    key=s3cfg['access_key'],
    client_kwargs={'endpoint_url': 'http://hackathon-o.s3.jc.rl.ac.uk'}
)
jasmin_s3.ls('sim-data')

['sim-data/5km-RAL3', 'sim-data/DYAMOND3_example_data']

In [24]:
url_tpl = 'http://hackathon-o.s3.jc.rl.ac.uk/sim-data/5km-RAL3/dev/data.2d.v1.z{zoom}.zarr'

ds = xr.open_zarr(url_tpl.format(zoom=10))

In [7]:
ds.data_vars

Data variables:
    clivi    (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    clt      (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    clwvi    (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    hflsd    (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    hfssd    (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    huss     (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    pr       (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    prs      (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    prw      (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ps       (time, cell) float32 528GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    psl      (time, cell) float32 528GB dask.array<chunksize=(1,

In [5]:
ds9 = ds.coarsen(cell=4).mean()

In [6]:
ds9.data_vars

Data variables:
    clivi    (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    clt      (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    clwvi    (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    hflsd    (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    hfssd    (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    huss     (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    pr       (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    prs      (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    prw      (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    ps       (time, cell) float32 132GB dask.array<chunksize=(1, 262144), meta=np.ndarray>
    psl      (time, cell) float32 132GB dask.array<chunksize=(1, 262144), 

In [39]:
chunks = (4, 262144)
preferred_chunks = {'time': 4, 'cell': 262144}
for da in ds9.data_vars.values():
    da.attrs['healpix_zoom'] = 9
    da.encoding['chunks'] = chunks
    da.encoding['preferred_chunks'] = preferred_chunks
    

In [29]:
store = s3fs.S3Map(root='s3://sim-data/5km-RAL3/dev/data.2d.v1.z{zoom}.zarr'.format(zoom=9), s3=jasmin_s3, check=False)

In [27]:
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [28]:
ds9.chunk({'time': 4, 'cell': 262144}).to_zarr(store, region={'time': slice(None), 'cell': slice(None)})

KeyboardInterrupt: 

In [49]:
store = s3fs.S3Map(root='s3://sim-data/5km-RAL3/dev/data.2d.v1.z{zoom}.coarsened.short2.zarr'.format(zoom=9), s3=jasmin_s3, check=False)

In [50]:
from dask.distributed import progress

In [51]:
ds9_short = ds9[['pr', 'tas']].isel(time=slice(8))
job = ds9_short.chunk({'time': 4, 'cell': 262144}).to_zarr(store, compute=False)
job.persist()
progress(job)

/tmp/ipykernel_726/3741500857.py:2: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  job = ds9_short.chunk({'time': 4, 'cell': 262144}).to_zarr(store, compute=False)


[########################################] | 100% Completed | 4.18 sms


ValueError: No clients found
Start a client and point it to the scheduler address
  from distributed import Client
  client = Client('ip-addr-of-scheduler:8786')


In [38]:
ds9.rsutcs.encoding

{'chunks': (1, 1048576),
 'preferred_chunks': {'time': 1, 'cell': 1048576},
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 '_FillValue': nan,
 'dtype': dtype('float32')}

In [14]:
ds9_tpl = xr.open_zarr(url_tpl.format(zoom=9))

In [15]:
ds9_tpl

<xarray.Dataset> Size: 3TB
Dimensions:  (cell: 3145728, time: 10489)
Coordinates:
  * cell     (cell) int64 25MB 0 1 2 3 4 ... 3145724 3145725 3145726 3145727
  * time     (time) datetime64[ns] 84kB 2020-01-20 ... 2021-04-01
Data variables: (12/24)
    clivi    (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    clt      (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    clwvi    (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    hflsd    (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    hfssd    (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    huss     (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    ...       ...
    rsut     (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    rsutcs   (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    tas      (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    ts       (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    uas      (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    vas      (time, cell) float32 132GB dask.array<chunksize=(4, 262144), meta=np.ndarray>

In [19]:
from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

In [1]:
import sys
sys.path.insert(0, '../../scripts/convert_latlon_pp_to_hp_nc/')
from processing_config import processing_config, chunks2d

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [16]:
from pathlib import Path

from dask.diagnostics import ProgressBar
from loguru import logger
import s3fs
import xarray as xr

def coarsen_healpix_region(z, start_time_idx, chunks, n_tgt_time_chunks=4):
    s3cfg = dict([l.split(' = ') for l in Path('/home/users/mmuetz/.s3cfg').read_text().split('\n') if l])

    jasmin_s3 = s3fs.S3FileSystem(
        anon=False, secret=s3cfg['secret_key'],
        key=s3cfg['access_key'],
        client_kwargs={'endpoint_url': 'http://hackathon-o.s3.jc.rl.ac.uk'}
    )

    z_tgt = z - 1
    chunks_src = chunks[z]
    chunks_tgt = chunks[z_tgt]
    n_times = chunks_tgt[0] * n_tgt_time_chunks

    # url_tpl = 'http://hackathon-o.s3.jc.rl.ac.uk/sim-data/5km-RAL3/dev/data.2d.v1.z{zoom}.zarr'
    store = s3fs.S3Map(root='s3://sim-data/5km-RAL3/dev/data.2d.v1.z{zoom}.zarr'.format(zoom=z), s3=jasmin_s3, check=False)
    store_tgt = s3fs.S3Map(root='s3://sim-data/5km-RAL3/dev/data.2d.v1.z{zoom}.zarr'.format(zoom=z_tgt), s3=jasmin_s3, check=False)

    logger.info('opening input zarr store')
    zarr_chunks = {'time': chunks_tgt[0], 'cell': -1}
    logger.debug(zarr_chunks)
    # ds_src = xr.open_dataset(store, chunks=zarr_chunks, engine='zarr').isel(time=slice(start_time_idx, start_time_idx + n_time_chunks))
    ds_src = xr.open_zarr(store, chunks=zarr_chunks).isel(time=slice(start_time_idx, start_time_idx + n_times))
    ds_src = ds_src[['pr', 'tas']]
    logger.debug(ds_src)

    logger.info('lazy coarsen')
    ds_tgt = ds_src.coarsen(cell=4).mean()
    logger.debug(ds_tgt)

    logger.info('modify metadata')
    preferred_chunks = {'time': chunks_tgt[0], 'cell': chunks_tgt[1]}
    for da in ds_tgt.data_vars.values():
        da.attrs['healpix_zoom'] = z_tgt
        da.encoding['chunks'] = chunks_tgt
        da.encoding['preferred_chunks'] = preferred_chunks

    logger.info('shorten coarse ds')

    logger.info('lazy to_zarr')
    region = {'time': slice(start_time_idx, start_time_idx + n_times), 'cell': slice(None)}
    logger.debug(region)
    job = ds_tgt.chunk(preferred_chunks).to_zarr(store_tgt, region=region, compute=False)

    logger.info('compute to_zarr')
    with ProgressBar() as pbar:
        job.compute()

In [20]:
coarsen_healpix_region(10, 0, chunks2d, 16)

2025-04-04 12:36:02.147 | INFO     | __main__:coarsen_healpix_region:26 - opening input zarr store
2025-04-04 12:36:02.148 | DEBUG    | __main__:coarsen_healpix_region:28 - {'time': 4, 'cell': -1}
2025-04-04 12:36:03.592 | DEBUG    | __main__:coarsen_healpix_region:32 - <xarray.Dataset> Size: 7GB
Dimensions:  (time: 64, cell: 12582912)
Coordinates:
  * cell     (cell) int64 101MB 0 1 2 3 ... 12582908 12582909 12582910 12582911
  * time     (time) datetime64[ns] 512B 2020-01-20 ... 2020-01-22T15:00:00
Data variables:
    pr       (time, cell) float32 3GB dask.array<chunksize=(4, 12582912), meta=np.ndarray>
    tas      (time, cell) float32 3GB dask.array<chunksize=(4, 12582912), meta=np.ndarray>
2025-04-04 12:36:03.597 | INFO     | __main__:coarsen_healpix_region:34 - lazy coarsen
2025-04-04 12:36:03.690 | DEBUG    | __main__:coarsen_healpix_region:36 - <xarray.Dataset> Size: 2GB
Dimensions:  (time: 64, cell: 3145728)
Coordinates:
  * cell     (cell) float64 25MB 1.5 5.5 9.5 ... 1.258e+

In [21]:
coarsen_healpix_region(9, 0, chunks2d, 4)

2025-04-04 12:36:37.188 | INFO     | __main__:coarsen_healpix_region:26 - opening input zarr store
2025-04-04 12:36:37.189 | DEBUG    | __main__:coarsen_healpix_region:28 - {'time': 16, 'cell': -1}
2025-04-04 12:36:38.186 | DEBUG    | __main__:coarsen_healpix_region:32 - <xarray.Dataset> Size: 2GB
Dimensions:  (time: 64, cell: 3145728)
Coordinates:
  * cell     (cell) int64 25MB 0 1 2 3 4 ... 3145724 3145725 3145726 3145727
  * time     (time) datetime64[ns] 512B 2020-01-20 ... 2020-01-22T15:00:00
Data variables:
    pr       (time, cell) float32 805MB dask.array<chunksize=(16, 3145728), meta=np.ndarray>
    tas      (time, cell) float32 805MB dask.array<chunksize=(16, 3145728), meta=np.ndarray>
2025-04-04 12:36:38.192 | INFO     | __main__:coarsen_healpix_region:34 - lazy coarsen
2025-04-04 12:36:38.236 | DEBUG    | __main__:coarsen_healpix_region:36 - <xarray.Dataset> Size: 409MB
Dimensions:  (time: 64, cell: 786432)
Coordinates:
  * cell     (cell) float64 6MB 1.5 5.5 9.5 ... 3.146e

In [22]:
coarsen_healpix_region(8, 0, chunks2d, 1)

2025-04-04 12:37:10.643 | INFO     | __main__:coarsen_healpix_region:26 - opening input zarr store
2025-04-04 12:37:10.644 | DEBUG    | __main__:coarsen_healpix_region:28 - {'time': 64, 'cell': -1}
2025-04-04 12:37:11.856 | DEBUG    | __main__:coarsen_healpix_region:32 - <xarray.Dataset> Size: 409MB
Dimensions:  (time: 64, cell: 786432)
Coordinates:
  * cell     (cell) int64 6MB 0 1 2 3 4 5 ... 786427 786428 786429 786430 786431
  * time     (time) datetime64[ns] 512B 2020-01-20 ... 2020-01-22T15:00:00
Data variables:
    pr       (time, cell) float32 201MB dask.array<chunksize=(64, 786432), meta=np.ndarray>
    tas      (time, cell) float32 201MB dask.array<chunksize=(64, 786432), meta=np.ndarray>
2025-04-04 12:37:11.860 | INFO     | __main__:coarsen_healpix_region:34 - lazy coarsen
2025-04-04 12:37:11.874 | DEBUG    | __main__:coarsen_healpix_region:36 - <xarray.Dataset> Size: 102MB
Dimensions:  (time: 64, cell: 196608)
Coordinates:
  * cell     (cell) float64 2MB 1.5 5.5 9.5 ... 7.8

In [23]:
coarsen_healpix_region(7, 0, chunks2d, 1)

2025-04-04 12:37:26.454 | INFO     | __main__:coarsen_healpix_region:26 - opening input zarr store
2025-04-04 12:37:26.455 | DEBUG    | __main__:coarsen_healpix_region:28 - {'time': 64, 'cell': -1}
2025-04-04 12:37:27.210 | DEBUG    | __main__:coarsen_healpix_region:32 - <xarray.Dataset> Size: 102MB
Dimensions:  (time: 64, cell: 196608)
Coordinates:
  * cell     (cell) int64 2MB 0 1 2 3 4 5 ... 196603 196604 196605 196606 196607
  * time     (time) datetime64[ns] 512B 2020-01-20 ... 2020-01-22T15:00:00
Data variables:
    pr       (time, cell) float32 50MB dask.array<chunksize=(64, 196608), meta=np.ndarray>
    tas      (time, cell) float32 50MB dask.array<chunksize=(64, 196608), meta=np.ndarray>
2025-04-04 12:37:27.213 | INFO     | __main__:coarsen_healpix_region:34 - lazy coarsen
2025-04-04 12:37:27.221 | DEBUG    | __main__:coarsen_healpix_region:36 - <xarray.Dataset> Size: 26MB
Dimensions:  (time: 64, cell: 49152)
Coordinates:
  * cell     (cell) float64 393kB 1.5 5.5 9.5 ... 1.966

In [26]:
coarsen_healpix_region(6, 0, chunks2d, 1)

2025-04-04 12:54:54.796 | INFO     | __main__:coarsen_healpix_region:26 - opening input zarr store
2025-04-04 12:54:54.797 | DEBUG    | __main__:coarsen_healpix_region:28 - {'time': 64, 'cell': -1}
2025-04-04 12:54:55.162 | DEBUG    | __main__:coarsen_healpix_region:32 - <xarray.Dataset> Size: 26MB
Dimensions:  (time: 64, cell: 49152)
Coordinates:
  * cell     (cell) int64 393kB 0 1 2 3 4 5 ... 49147 49148 49149 49150 49151
  * time     (time) datetime64[ns] 512B 2020-01-20 ... 2020-01-22T15:00:00
Data variables:
    pr       (time, cell) float32 13MB dask.array<chunksize=(64, 49152), meta=np.ndarray>
    tas      (time, cell) float32 13MB dask.array<chunksize=(64, 49152), meta=np.ndarray>
2025-04-04 12:54:55.165 | INFO     | __main__:coarsen_healpix_region:34 - lazy coarsen
2025-04-04 12:54:55.171 | DEBUG    | __main__:coarsen_healpix_region:36 - <xarray.Dataset> Size: 6MB
Dimensions:  (time: 64, cell: 12288)
Coordinates:
  * cell     (cell) float64 98kB 1.5 5.5 9.5 ... 4.914e+04 4.91

In [24]:
chunks2d

{10: (1, 1048576),
 9: (4, 262144),
 8: (16, 65536),
 7: (64, 16384),
 6: (64, 16384),
 5: (64, 12288),
 4: (256, 3072),
 3: (1024, 768),
 2: (4096, 192),
 1: (16384, 48),
 0: (65536, 12)}